# Data Load

In [1]:
import config
from pshmodule.utils import filemanager as fm

In [2]:
df = fm.load(config.temp_ref_data)

extension : .pickle
Loaded 180940 records from /Volumes/GoogleDrive/내 드라이브/MemeProject/data/temp_for_ref.pickle


In [3]:
df.head()

,u,cpr,mm,cpo
0,나 중간고사 반에서 1등했어,그렇다면 통 크게 함 쏴야지!,근데 이제 내 것도 곁들인...,NaN
1,나 중간고사 반에서 1등했어,그렇다면 통 크게 함 쏴야지!,근데 이제 내 몫을 제일 많이 곁들인...,NaN
2,나 중간고사 반에서 1등했어,그렇다면 통 크게 함 쏴야지!,근데 이제 대춘이 몫도 곁들인...,NaN
3,나 중간고사 반에서 1등했어,그렇다면 통 크게 함 쏴야지!,근데 이제 배보다 배꼽이 더 큰 걸 곁들인...,NaN
4,나 중간고사 반에서 1등했어,그렇다면 통 크게 함 쏴야지!,근데 이제 과소비를 곁들인...,NaN


# 

# Train

In [4]:
import math
import logging
from datetime import datetime

import torch
from torch.utils.data import DataLoader
from sentence_transformers import SentenceTransformer,  LoggingHandler, losses, models, util
from sentence_transformers.evaluation import EmbeddingSimilarityEvaluator
from sentence_transformers.readers import InputExample